In [ ]:
#TODO: 1. optimize parameters for YOLO nano/small/medium model

# install ultralytics, import yolo
!pip install ultralytics
!pip install easyocr
from ultralytics import YOLO
import cv2
import numpy as np
from google.colab import files
#from sort.sort import *
import easyocr

In [ ]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available and being used

True


In [ ]:
# upload data from https://www.pexels.com/video/traffic-flow-in-the-highway-2103099/
data = files.upload()
print("Data are uploaded")

Data are uploaded


In [ ]:
# import yolo
model = YOLO('yolov8n.pt').to('cuda' if torch.cuda.is_available() else 'cpu')
plate_detector_model = YOLO('./license_plate_detector.pt')
ocr_reader = easyocr.Reader(['en'])

In [ ]:
from ultralytics import YOLO
import cv2
from google.colab import files

# Upload video file
print("Please upload your traffic video file...")

video_path = '/content/2103099-hd_1920_1080_30fps.mp4'
print(f"✅ Uploaded: {video_path}")

# Load YOLO model (using nano version for faster processing)
car_model = YOLO('yolov8n.pt')  # or 'yolov8s.pt' for better balance of speed/accuracy

output_video_path = "car_detection_output.mp4"

# Load video
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Prepare video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Detection data storage
detection_data = []

# we will use this to track all the vehicles
#tracker = Sort()

# Car detection parameters
conf_threshold = 0.3  # Confidence threshold for car detection
iou_threshold = 0.5   # IOU threshold for NMS
box_thickness = 2      # Bounding box thickness
box_color = (0, 255, 0)  # Green color for car boxes

while cap.isOpened(): # read frames
    ret, frame = cap.read()
    if not ret:
        break

    # THIS IS DETECTION
    results = car_model(frame, conf=conf_threshold, iou=iou_threshold)

    frame_data = {
        "frame_number": int(cap.get(cv2.CAP_PROP_POS_FRAMES)),
        "cars": []
    }

    # Process detections
    for result in results:
        for box in result.boxes:
            if int(box.cls) in [2, 3, 5, 7]: # Car, Motorbike, Bus, Truck
                bbox = box.xyxy[0].tolist() # Save x y x y of bounding box
                confidence = float(box.conf) # Save confidence

                # Store car data
                frame_data["cars"].append({
                    "bbox": bbox,
                    "confidence": confidence
                })

                license_plates = plate_detector_model(frame)[0]
                for license_plate in license_plates.boxes.data.tolist():
                    lx1, ly1, lx2, ly2, score, class_id = license_plate #get licence plate boundaries

                # Crop license plate and apply filter
                license_plate_crop = frame[int(ly1):int(ly2), int(lx1): int(lx2), :]
                license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2RGB)
                _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)
                # takes everything above 64 and sets it to 0 , takes everything below 64 and sets it to 255

                # OCR
                license_plate = ""
                ocr_results = ocr_reader.readtext(license_plate_crop_thresh)
                if ocr_results:
                  license_plate = ocr_results[0][1]

                # Draw car bounding box
                x1, y1, x2, y2 = map(int, bbox)
                cv2.rectangle(frame,
                              (x1, y1),
                              (x2, y2),
                              box_color,
                              box_thickness)

                # Draw licence bounding box
                cv2.rectangle(frame, (int(lx1), int(ly1)), (int(lx2), int(ly2)), box_color, 2)
                label = f"Car {confidence:.2f}"
                if license_plate:
                    label += f" | LP: {license_plate}"
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 1)

                # Add confidence text
#                cv2.putText(frame,
#                           f"Car: {confidence:.2f}",
#                           (x1, y1 - 10),
#                           cv2.FONT_HERSHEY_SIMPLEX,
#                           0.5,
#                           box_color,
#                           1)

    detection_data.append(frame_data)
    out.write(frame)

cap.release()
out.release()

print(f"✅ Car detection complete! Output saved as {output_video_path}")
print(f"✅ Detected {sum(len(frame['cars']) for frame in detection_data)} cars total")
print(f"✅ Detection data contains {len(detection_data)} frames")

# For future license plate detection:
print("\nNext steps:")
print("1. Get a license plate detection model")
print("2. Add the license plate detection code where indicated")
print("3. The current data structure is already prepared for LP detection results")

Streaming output truncated to the last 5000 lines.

0: 384x640 2 license_plates, 9.9ms
Speed: 3.1ms preprocess, 9.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 9.9ms
Speed: 3.4ms preprocess, 9.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 9.2ms
Speed: 3.3ms preprocess, 9.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 8.0ms
Speed: 3.2ms preprocess, 8.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 8.6ms
Speed: 3.2ms preprocess, 8.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 7.6ms
Speed: 4.2ms preprocess, 7.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 8.6ms
Speed: 3.1ms preprocess, 8.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate,

In [ ]:
files.download(output_video_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>